# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、人間による好みの予測に関する問題に取り組んでいます。具体的には、二つのチャットボットが生成した応答のどちらが好まれるかを予測するための確率を算出し、提出用のCSVファイルを作成します。

ノートブックでは、以下の手法とライブラリが使用されています：

1. **ライブラリのインポート**: NumPyとPandasを使用してデータを処理し、Scikit-learnから対数損失（log loss）をインポートしています。

2. **データの読み込み**: Kaggleから提供されたテストデータセット（`test.csv`）をPandasを用いて読み込んでいます。

3. **特徴量の生成**:
   - 各応答（`response_a`と`response_b`）の文字数を計算し、それぞれ新しいカラム（`response_a_len`と`response_b_len`）に追加します。
   - その後、応答の長さの比率（`ratio_response_b_response_a`）を計算し、新たなカラムに追加します。

4. **確率の割り当て**:
   - 訓練データセットから事前に計算された確率の平均値（`avg_1`, `avg_2`, `avg_3`）を使用して、特定のしきい値に基づいて各行に勝者モデルの確率（モデルAの勝率、モデルBの勝率、引き分けの確率）を割り当てる関数を定義しています。

5. **提出用データフレームの作成**: 確率値を含む新しいデータフレームを作成し、元のデータから`id`カラムを取得して提出用の形式に整えています。

6. **CSVファイルの作成**: 最後に、作成したデータフレームを`submission.csv`というファイル名で保存します。

このNotebookは、特徴量エンジニアリングとしきい値を指定した条件分岐を通じて、チャットボットの応答に基づく好みの予測を行う手法を示しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語や概念の簡単な解説をリスト化しました。特にこのJupyter NotebookやLMSYS - Chatbot Arenaコンペティションに特有の内容に焦点を当てています。

1. **カラム (Column)**:
   - データフレーム（テーブル型のデータ構造）内の特定のフィールドを指します。例えば、このNotebookでは、`response_a` や `response_b` などがカラムとなっています。

2. **応答 (Response)**:
   - チャットボットにおいてユーザーの質問やプロンプトに対して生成される返答のことです。本コンペティションでは、異なるモデル（チャットボット）が生成した二つの応答を比較します。

3. **長さ (Length)**:
   - テキストデータにおいて、文字数を数えること。ここでは、各応答の長さを計算し、データフレームに新しいカラム（`response_a_len` や `response_b_len`）を追加しています。

4. **比率 (Ratio)**:
   - 二つの量の関係を示す数値。ここでは、`response_b` の長さを `response_a` の長さで割った比率を求め、応答同士の相対的な長さを比較しています。

5. **しきい値 (Threshold)**:
   - 特定の条件を満たすかどうかを判断するための基準値のこと。ここでは、比率に応じて異なる確率を割り当てる際の基準として使用されています。

6. **確率 (Probability)**:
   - 何かが起こる可能性を示す数値で、0から1の間の値を取ります。このNotebookでは、どのモデルが優れているか（あるいは引き分けか）を表すための確率が設定されます。

7. **applyメソッド**:
   - Pandasのデータフレームにおいて、指定した関数を各行または各列に適用するためのメソッドです。このNotebookでは、`assign_prob` 関数をデータフレームの各行に適用して確率を割り当てています。

8. **データフレーム (DataFrame)**:
   - Pandasライブラリで使用される二次元のサイズ可変なデータ構造で、異なるデータ型のカラムを持つことができます。表形式のデータを扱う際によく利用されます。

9. **CSVファイル (Comma-Separated Values)**:
   - コンマ区切りの形式でデータを保存するファイル。データの扱いがシンプルで、他のプログラムでも簡単に読み込めることから、データの入出力に広く使用されます。ここでは、最終的な提出用ファイルを作成するために使用されています。

10. **ロス関数 (Loss Function)**:
    - モデルの予測と実際の結果との誤差を測る指標。評価段階においてモデルの性能を数値的に表現するためのもので、ここでは予測の精度を示すために `log_loss` が使われています。

これらの用語は、初心者が理解を深めるために重要です。特に、コンペティションの目的やデータの処理方法を把握するのに役立ちます。

---


In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import log_loss
# データセットを読み込む
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# response_a の長さを計算して新しいカラムに追加
df['response_a_len'] = df['response_a'].apply(lambda x: len(x))
# response_b の長さを計算して新しいカラムに追加
df['response_b_len'] = df['response_b'].apply(lambda x: len(x))

# response_b と response_a の長さの比率を計算して新しいカラムに追加
df["ratio_response_b_response_a"] = df['response_b_len'] / df['response_a_len']

# これは訓練データセットから計算した値
avg_1 = {'winner_model_a': 0.2550469811145223, 'winner_model_b': 0.4547399758117034, 'winner_tie': 0.2902130430737743}
avg_2 = {'winner_model_a': 0.31478770131771594, 'winner_model_b': 0.33601756954612005, 'winner_tie': 0.349194729136164}
avg_3 = {'winner_model_a': 0.45816715010877446, 'winner_model_b': 0.25086113125453224, 'winner_tie': 0.29097171863669324}

# 以下のしきい値も訓練データセットから計算したもの
def assign_prob(row):
    # 比率がある閾値以下の場合は avg_3 の値を返す
    if row['ratio_response_b_response_a'] <= 0.838296:
        return avg_3.values()
    # 比率が別の閾値以下の場合は avg_2 の値を返す
    elif row['ratio_response_b_response_a'] <= 1.004711:
        return avg_2.values()
    # それ以外の場合は avg_1 の値を返す
    else:
        return avg_1.values()

# 各行に対して確率を割り当てる
df[['winner_model_a', 'winner_model_b', 'winner_tie']] = df.apply(
    lambda x: assign_prob(x), axis=1, result_type='expand'
)

In [ ]:
# 提出用のデータフレームを作成
submission_df = pd.DataFrame(
    {
        'id': df['id'],  # 元のデータから id カラムを取得
        'winner_model_a': df['winner_model_a'],  # モデル A の勝者確率を取得
        'winner_model_b': df['winner_model_b'],  # モデル B の勝者確率を取得
        'winner_tie': df['winner_tie']  # 引き分け確率を取得
    }
)

# 提出用の CSV ファイルを作成
submission_df.to_csv("submission.csv", index=False)

# 作成したデータフレームを表示
submission_df